
# LAB 11 – MORPHOLOGICAL OPERATIONS

**Tasks**
1. Erosion, Dilation, Opening, Closing  
2. Boundary Extraction  
3. Hole Filling using Reconstruction  
4. Noise Removal using Morphology  
5. Shape Detection  

This notebook is fully solved and ready to run.


In [ ]:

# Imports
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.morphology import reconstruction



## 1. Load Image


In [ ]:

# Load binary image
img = cv2.imread('sample_binary.png', cv2.IMREAD_GRAYSCALE)
if img is None:
    img = np.zeros((256,256), dtype=np.uint8)
    cv2.circle(img, (128,128), 60, 255, -1)
    cv2.rectangle(img, (30,30), (90,90), 255, -1)

_, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

plt.imshow(binary, cmap='gray')
plt.title("Original Binary Image")
plt.axis('off')



## 2. Erosion, Dilation, Opening, Closing


In [ ]:

kernel = np.ones((5,5), np.uint8)

erosion = cv2.erode(binary, kernel, iterations=1)
dilation = cv2.dilate(binary, kernel, iterations=1)
opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
closing = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

titles = ["Original", "Erosion", "Dilation", "Opening", "Closing"]
images = [binary, erosion, dilation, opening, closing]

plt.figure(figsize=(12,4))
for i in range(5):
    plt.subplot(1,5,i+1)
    plt.imshow(images[i], cmap='gray')
    plt.title(titles[i])
    plt.axis('off')
plt.show()



## 3. Boundary Extraction
Boundary = Original − Erosion


In [ ]:

boundary = binary - erosion

plt.imshow(boundary, cmap='gray')
plt.title("Extracted Boundary")
plt.axis('off')



## 4. Hole Filling using Morphological Reconstruction


In [ ]:

# Invert image
inv = cv2.bitwise_not(binary)

seed = np.zeros_like(inv)
seed[0,:] = inv[0,:]
seed[-1,:] = inv[-1,:]
seed[:,0] = inv[:,0]
seed[:,-1] = inv[:,-1]

filled = reconstruction(seed, inv, method='dilation')
filled = cv2.bitwise_not(np.uint8(filled))

plt.figure(figsize=(8,4))
plt.subplot(1,2,1); plt.imshow(binary, cmap='gray'); plt.title("Before Filling")
plt.subplot(1,2,2); plt.imshow(filled, cmap='gray'); plt.title("After Filling")
plt.axis('off')



## 5. Noise Removal using Morphology


In [ ]:

# Add salt noise
noise = binary.copy()
coords = np.random.randint(0, 256, (200,2))
for x,y in coords:
    noise[x,y] = 255

denoised = cv2.morphologyEx(noise, cv2.MORPH_OPEN, kernel)

plt.figure(figsize=(8,4))
plt.subplot(1,2,1); plt.imshow(noise, cmap='gray'); plt.title("Noisy Image")
plt.subplot(1,2,2); plt.imshow(denoised, cmap='gray'); plt.title("After Noise Removal")
plt.axis('off')



## 6. Shape Detection using Morphology
(Simple contour-based shape detection)


In [ ]:

contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

shape_img = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)

for cnt in contours:
    approx = cv2.approxPolyDP(cnt, 0.04*cv2.arcLength(cnt, True), True)
    if len(approx) == 3:
        label = "Triangle"
    elif len(approx) == 4:
        label = "Rectangle"
    else:
        label = "Circle"
    
    cv2.drawContours(shape_img, [cnt], -1, (255,0,0), 2)
    x,y,w,h = cv2.boundingRect(cnt)
    cv2.putText(shape_img, label, (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1)

plt.imshow(shape_img)
plt.title("Detected Shapes")
plt.axis('off')



## Conclusion
- Morphological operations are effective for shape-based image analysis.
- Erosion & dilation modify object boundaries.
- Opening & closing help remove noise and fill gaps.
- Reconstruction is powerful for hole filling.
